In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data = data.drop(["RowNumber", "CustomerId", "Surname"], axis=1)


In [3]:
label_encoder_gender = LabelEncoder()
data["Gender"] = label_encoder_gender.fit_transform(data["Gender"])


In [4]:
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
geo_encoded = one_hot_encoder.fit_transform(data[["Geography"]]).toarray()
geo_encoded_tf = pd.DataFrame(geo_encoded, columns=one_hot_encoder.get_feature_names_out(['Geography']))

In [5]:
geo_encoded_tf

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [6]:
data = pd.concat([data.drop("Geography", axis=1), geo_encoded_tf], axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [7]:
X = data.drop("EstimatedSalary", axis=1)
y = data["EstimatedSalary"]

In [8]:
with open("label_encoder_gender_2.pkl", "wb") as file:
    pickle.dump(label_encoder_gender, file)

with open("onehot_encoder_geo_2.pkl", "wb") as file:
    pickle.dump(one_hot_encoder, file)

In [9]:
##Split the data in training and testing set
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

##Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
with open("scaler_2.pkl", "wb") as file:
    pickle.dump(scaler, file)

In [24]:
#ANN Regression

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import datetime

In [25]:
model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)), ##HL1
    Dense(32, activation="relu"), ##HL2
    Dense(1)
])

In [26]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                832       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), 
    loss="mean_absolute_error", 
    metrics=["mae"]
)

In [32]:
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100
)

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 48987.3945 - mae: 48987.3945 - val_loss: 50466.4062 - val_mae: 50466.4062
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 48941.4922 - mae: 48941.4922 - val_loss: 50581.5000 - val_mae: 50581.5000
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 48945.8633 - mae: 48945.8633 - val_loss: 50581.7812 - val_mae: 50581.7812
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 48941.5547 - mae: 48941.5547 - val_loss: 50639.7461 - val_mae: 50639.7461
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 48936.4727 - mae: 48936.4727 - val_loss: 50540.1914 - val_mae: 50540.1914
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 48916.9219 - mae: 48916.9219 - val_loss: 50592.9219 - val_mae: 50592.9219
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 48910.5039 - mae: 48910.5039 - 

In [ ]:
model.save("regression-model.h5")